# Assigment 5

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Generate a function called **import_data_sbs_banks** that take as arguments `start_month`, `start_year`, `end_month` and `end_year`. This function should be able to import the files in **..\_data\sbs\B_RawData\bancos** taking into consideration these arguments. Additionally, it should work if it is only given the first two arguments `(start_mont, start_year)`. The function should `returns` a nested dictionary. The `main key` must be year and `second key` must be `month`. The values should be the dataframes.

In [23]:
import os
import pandas as pd
from pathlib import Path

def import_data_sbs_banks(start_month, start_year, end_month=None, end_year=None):
    # Define path
    datasbs_path = Path("../../_data/sbs/B_RawData/bancos")

    # Read files using the path and store them in a list
    list1 = [file for file in datasbs_path.glob("*.xlsx")]

    # Create an empty dictionary to store the dataframes
    dataframe_dict = {}

    # Auxiliary function to extract the date from a file
    def get_file_date(file):
        return pd.to_datetime(file.stem[len("table_clean_"):], format="%d_%m_%Y")

    # Iterate over each file in the list
    for file in list1:
        # Read Excel file and store temporarily
        tmp = pd.read_excel(file)

        # Assign the date to the dataframe
        tmp["date_info"] = get_file_date(file)

        # Store in the previously created empty dictionary
        dataframe_dict[file] = tmp

    # Define magic values as constants
    FILE_PREFIX_LEN = len("table_clean_")

    # Conditional to make it work even if only 2 arguments are provided
    if end_month is None:
        end_month = start_month
    if end_year is None:
        end_year = start_year

    # Create start_date and end_date arguments
    start_date = pd.to_datetime(f"{start_month}/01/{start_year}", format="%m/%d/%Y")

    if end_month == 12:
        end_date = pd.to_datetime(f"01/01/{end_year + 1}", format="%m/%d/%Y")
    else:
        end_date = pd.to_datetime(f"{end_month + 1}/01/{end_year}", format="%m/%d/%Y")

    # Select the dataframes that are between the specified dates
    selected_dfs = {
        k: v for k, v in dataframe_dict.items()
        if start_date <= pd.Timestamp(v["date_info"].iloc[0]) <= end_date
    }

    # Sort the selected dataframes according to their date
    sorted_dfs = sorted(
        selected_dfs.items(),
        key=lambda x: (
            x[1]["date_info"].dt.year.values[0],
            x[1]["date_info"].dt.month.values[0],
        ),
    )

    # Create an empty dictionary for the nested dataframes
    dictionary_sbs_banks = {}

    # Use 2 keys: month and then year for the sorted dataframes
    # Specify the final structure of the dictionary
    for key, value in sorted_dfs:
        year = value["date_info"].dt.year.values[0]
        month = value["date_info"].dt.month.values[0]
        dictionary_sbs_banks.setdefault(year, {})[month] = value

    return dictionary_sbs_banks


In [27]:
# import_data_sbs_banks(1, 2004, 12, 2005)
 
# Works with start_month and start_year 
import_data_sbs_banks(1, 2004)



{2004: {1:               Unnamed: 0                 COMERCIAL COMERCIAL.1  \
  0                    NaN  Avances en Cta.Corriente  Sobregiros   
  1                    NaN  Avances en Cta.Corriente  Sobregiros   
  2                TRABAJO                         -           -   
  3                   BBVA                     38.94       48.85   
  4      BANCO DE COMERCIO                         -      140.34   
  5       BANCO DE CREDITO                     70.12       87.57   
  6        BANCO PICHINCHA                    113.09           -   
  7                 BANBIF                     42.79       47.52   
  8     STANDARD CHARTERED                     18.75       25.00   
  9     BANCO SUDAMERICANO                      4.79       40.55   
  10       WIESE SUDAMERIS                         -           -   
  11           BANK BOSTON                     14.10       82.12   
  12        BNP PARIBAS EL                         -       30.00   
  13     CITIBANK DEL PERU             